#This is an example of performing Video Activity Recognition using LSTM
Modified from "Hands-on Computer Vision with TensorFlow 2" by B. Planche and E. Andres

In [9]:
# STEP 1:  Install packages in the current environment
import sys
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install scikit-learn

In [2]:
#!{sys.executable} -m pip install tensorflow==1.14
!{sys.executable} -m pip install tensorflow==2.0

#Step 2: import modules

In [10]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-79673bd9355c>", line 1, in <module>
    import tensorflow as tf
  File "C:\Users\STSC.LNVO-126908.000\AppData\Roaming\Python\Python37\site-packages\tensorflow\__init__.py", line 98, in <module>
    from tensorflow_core import *
  File "C:\Users\STSC.LNVO-126908.000\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\__init__.py", line 42, in <module>
    from . _api.v2 import audio
ModuleNotFoundError: No module named 'tensorflow_core._api.v2'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundE

ModuleNotFoundError: No module named 'tensorflow_core._api.v2'

In [2]:
#tf.enable_eager_execution()
#tf.executing_eagerly()

#Step 3: setup variables

In [11]:
#location of where YOU have installed the data set UCF-101 located
# at 
#BASE_PATH = '../data/UCF-101'
#change the base path to location YOU installed UCF-101 dataset
#BASE_PATH = 'C:/Grewe/Classes/CS663/Mat/LSTM/data/UCF-101'
BASE_PATH = 'C:\\Users\\STSC.LNVO-126908.000\\Desktop\\FallDetection\\Classes\\'
VIDEOS_PATH = os.path.join(BASE_PATH, '**','*.mp4')

#this specifies the sequence length will process by LSTM
SEQUENCE_LENGTH = 40
BATCH_SIZE = 16
print(VIDEOS_PATH)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-b7ec6b68b2fc>", line 7, in <module>
    VIDEOS_PATH = os.path.join(BASE_PATH, '**','*.mp4')
NameError: name 'os' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, 

NameError: name 'os' is not defined

### STEP 4:  sample the video --do not process every frame
PART 1: define function frame_generator() that creates Sequence_length samples by taking every Kth sample were K= num_frames_in_video / SEQUENCE LENGTH     PART 2: you load the DataSet and specify the output will be frames of size 299x299 x3(rgb) AND you create batches of 16 together at a time

In [4]:
def frame_generator():
    video_paths = tf.io.gfile.glob(VIDEOS_PATH)
    np.random.shuffle(video_paths)
    for video_path in video_paths:
        frames = []
        cap = cv2.VideoCapture(video_path)
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_every_frame = max(1, num_frames // SEQUENCE_LENGTH)
        current_frame = 0
        
        label = os.path.basename(os.path.dirname(video_path))
        
        max_images = SEQUENCE_LENGTH
        while True:
            success, frame = cap.read()
            if not success:
                break
                
            if current_frame % sample_every_frame == 0:
                frame = frame[:, :, ::-1]
                img = tf.image.resize(frame, (224, 224))
                img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
                max_images -= 1
                yield img, video_path
                
            if max_images == 0:
                break
            current_frame += 1
            

In [5]:
dataset = tf.data.Dataset.from_generator(frame_generator,
                                         output_types=(tf.float32, tf.string),
                                         output_shapes=((224, 224, 3), ()))

dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [18]:
print(dataset)

<DatasetV1Adapter shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.string)>


### STEP 5: 
For Feature Extraction we are going to use a existing CNN model called Inception V3 which is built into TensorFlow

In [6]:
mobilenet_v2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
x = mobilenet_v2.output

# We add Average Pooling to transform the feature map from
# 8 * 8 * 2048 to 1 x 2048, as we don't need spatial information
pooling_output = tf.keras.layers.GlobalAveragePooling2D()(x)
feature_extraction_model = tf.keras.Model(mobilenet_v2.input,pooling_output)


### STEP 6: 
Extract Features using our InceptionV3 CNN model

In [7]:
#Don't run this for if done with feature extraction 
current_path = None
all_features = []

#cycle through the dataset and visit each image, note the tdqm is a progress bar
#that updates each time a new iteration is called 
#call feature_extraction_model above (Inception v3) for the image to extract the features
for img, batch_paths in tqdm.tqdm(dataset):
    batch_features = feature_extraction_model(img)
    #reshape the tensor 
    batch_features = tf.reshape(batch_features, 
                              (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if path != current_path and current_path is not None:
            output_path = current_path.decode().replace('.mp4', '.npy')
            np.save(output_path, all_features)
            all_features = []
            
        current_path = path
        all_features.append(features)

1308it [20:24,  1.07it/s]


### STEP 7: 
Create a MyLabelBinarizer for 2 Classes

In [8]:
#run this

class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y
    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)

In [9]:
#run this
LABELS = ['Falling','Walking']
encoder = MyLabelBinarizer()
encoder.fit(LABELS)
print(encoder.classes_)
print(encoder.transform(['Falling', 'Walking']))

t= encoder.transform(['Falling', 'Walking', 'Walking'])
print(t)
print(encoder.inverse_transform(t))
print("length of labrels " + str(len(LABELS)))

['Falling' 'Walking']
[[0 1]
 [1 0]]
[[0 1]
 [1 0]
 [1 0]]
['Falling' 'Walking' 'Walking']
length of labrels 2


### STEP 8: 
    Create the LSTM model:    1) Masking layer  2) LSTM layer with 512 cells, dropout 0.5, recurrent_dropout of 0.5  
 3) a fully connected relu activation layer with 256 outputs,  4) a droupout layer 0.5  5) a final decision fully connected layer of putput length of labels  (which is the number of classes) with softmax activation.

In [10]:
#setup a keras Sequential model with 1) Masking layer  2) LSTM layer with 512 cells, dropout 0.5, recurrent_dropout of 0.5  
# 3) a fully connected relu activation layer with 256 outputs,  4) a droupout layer 5) a final decision fully connected layer of length of labels
# (which is the number of classes) with softmax activation.
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.),
    tf.keras.layers.LSTM(512, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
    #tf.keras.layers.Dense(len(LABELS), activation='softmax')
])

### STEP 8: 
Setup for the model the Loss function, the Optimizer function, and any metrics want to compute in training

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

### STEP 9: 
Setup  the training and test list which are lists of the training filenames.   Note you will need to change the location of these files to point to your location.  Define a function make_generator that returns a generator which will randomly shuffle a file list (either training or testing that will be passed later) and then changes the file extension of the avi files listed in the list to .npy which is our features for that avi video which were calcluated in step 6

In [14]:
#train_file = '/Users/subhi/Downloads/CV-proj3/Data/trainlist.txt'
#test_file = '/Users/subhi/Downloads/CV-proj3/Data/testlist.txt'
test_file = 'C:/Users/STSC.LNVO-126908.000/Desktop/FallDetection/trainlist.txt'
train_file = 'C:/Users/STSC.LNVO-126908.000/Desktop/FallDetection/testlist.txt'

with open(test_file) as f:
    test_list = [row.strip() for row in list(f)]

with open(train_file) as f:
     train_list = [row.strip() for row in list(f)]
    #train_list = [row.split(' ')[0] for row in train_list]
#print(train_list)
def make_generator(file_list):
    def generator():
        np.random.shuffle(file_list)
        for path in file_list:
            full_path = os.path.join(BASE_PATH, path).replace('.mp4', '.npy')
            
            label = os.path.basename(os.path.dirname(path))
            features = np.load(full_path)
            
            
            padded_sequence = np.zeros((SEQUENCE_LENGTH, 1280))
            padded_sequence[0:len(features)] = np.array(features)
            
            transformed_label = encoder.transform([label])
            
            yield padded_sequence, transformed_label[0]
    return generator

In [15]:
print(train_list)

['Falling\\Falling_0.mp4', 'Falling\\Falling_1.mp4', 'Falling\\Falling_2.mp4', 'Falling\\Falling_3.mp4', 'Falling\\Falling_4.mp4', 'Falling\\Falling_5.mp4', 'Falling\\Falling_6.mp4', 'Falling\\Falling_7.mp4', 'Falling\\Falling_30.mp4', 'Falling\\Falling_31.mp4', 'Falling\\Falling_32.mp4', 'Falling\\Falling_33.mp4', 'Falling\\Falling_34.mp4', 'Falling\\Falling_35.mp4', 'Falling\\Falling_36.mp4', 'Falling\\Falling_37.mp4', 'Falling\\Falling_38.mp4', 'Falling\\Falling_39.mp4', 'Falling\\Falling_40.mp4', 'Falling\\Falling_41.mp4', 'Falling\\Falling_42.mp4', 'Falling\\Falling_43.mp4', 'Falling\\Falling_44.mp4', 'Falling\\Falling_45.mp4', 'Falling\\Falling_46.mp4', 'Falling\\Falling_47.mp4', 'Falling\\Falling_48.mp4', 'Falling\\Falling_49.mp4', 'Falling\\Falling_50.mp4', 'Falling\\Falling_51.mp4', 'Falling\\Falling_52.mp4', 'Falling\\Falling_53.mp4', 'Falling\\Falling_54.mp4', 'Falling\\Falling_55.mp4', 'Falling\\Falling_56.mp4', 'Falling\\Falling_57.mp4', 'Falling\\Falling_58.mp4', 'Falling

In [16]:
print(test_list)

['Falling\\Falling_0.mp4', 'Falling\\Falling_1.mp4', 'Falling\\Falling_2.mp4', 'Falling\\Falling_3.mp4', 'Falling\\Falling_4.mp4', 'Falling\\Falling_5.mp4', 'Falling\\Falling_6.mp4', 'Falling\\Falling_7.mp4', 'Falling\\Falling_8.mp4', 'Falling\\Falling_9.mp4', 'Falling\\Falling_10.mp4', 'Falling\\Falling_11.mp4', 'Falling\\Falling_12.mp4', 'Falling\\Falling_13.mp4', 'Falling\\Falling_14.mp4', 'Falling\\Falling_15.mp4', 'Falling\\Falling_16.mp4', 'Falling\\Falling_17.mp4', 'Falling\\Falling_18.mp4', 'Falling\\Falling_19.mp4', 'Falling\\Falling_20.mp4', 'Falling\\Falling_21.mp4', 'Falling\\Falling_22.mp4', 'Falling\\Falling_23.mp4', 'Falling\\Falling_24.mp4', 'Falling\\Falling_25.mp4', 'Falling\\Falling_26.mp4', 'Falling\\Falling_27.mp4', 'Falling\\Falling_28.mp4', 'Falling\\Falling_29.mp4', 'Falling\\Falling_30.mp4', 'Falling\\Falling_31.mp4', 'Falling\\Falling_32.mp4', 'Falling\\Falling_33.mp4', 'Falling\\Falling_34.mp4', 'Falling\\Falling_35.mp4', 'Falling\\Falling_36.mp4', 'Falling\\

### STEP 10: 
Setup the train_dataset and valid_dataset (validation/testing).   Here we setting up training batch sets of 16.  

In [17]:
#for tesnorflow 2.*
train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                output_types=(tf.float32, tf.int16),
                output_shapes=((SEQUENCE_LENGTH, 1280), (len(LABELS))))
                 

train_dataset = train_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, 1280), (len(LABELS))))
valid_dataset = valid_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [18]:
#for tensorflow 1.1.4
train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=(tf.TensorShape([SEQUENCE_LENGTH, 1280]), tf.TensorShape([len(LABELS)])))

train_dataset = train_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=(tf.TensorShape([SEQUENCE_LENGTH, 1280]), tf.TensorShape([len(LABELS)])))
valid_dataset = valid_dataset.batch(16,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [19]:
print(train_dataset)

<DatasetV1Adapter shapes: ((16, 40, 1280), (16, 2)), types: (tf.float32, tf.int16)>


In [20]:
print(valid_dataset)

<DatasetV1Adapter shapes: ((16, 40, 1280), (16, 2)), types: (tf.float32, tf.int16)>


In [35]:
BASE_DATA_PATH = 'C:/Users/STSC.LNVO-126908.000/Desktop/FallDetection'
mylog_dir = os.path.join( BASE_DATA_PATH, "train_log")
print("Mylog directory = " + mylog_dir)

Mylog directory = C:/Users/STSC.LNVO-126908.000/Desktop/FallDetection\train_log


In [37]:
#tf 2.0
tensorboard_callback = tf.keras.callbacks.TensorBoard(os.path.join('tmp'), update_freq=1000)
#model.fit(train_dataset, epochs=1, callbacks=[tensorboard_callback], validation_data=valid_dataset)
model.fit(train_dataset, epochs=1,validation_data=valid_dataset)

UnknownError: FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\STSC.LNVO-126908.000\\Desktop\\FallDetection\\Classes\\Falling\\Falling_479.npy'
Traceback (most recent call last):

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\tensorflow_core\python\ops\script_ops.py", line 221, in __call__
    ret = func(*args)

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 585, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-14-b441973f18d5>", line 20, in generator
    features = np.load(full_path)

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 422, in load
    fid = open(os_fspath(file), "rb")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\STSC.LNVO-126908.000\\Desktop\\FallDetection\\Classes\\Falling\\Falling_479.npy'


	 [[{{node PyFunc}}]] [Op:IteratorGetNextSync

In [28]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [30]:
#tf 1.1.4
model.fit(train_dataset,epochs=1, validation_data= valid_dataset, validation_steps=4,verbose=0)

UnknownError: FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\STSC.LNVO-126908.000\\Desktop\\FallDetection\\Classes\\Falling\\Falling_477.npy'
Traceback (most recent call last):

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\tensorflow_core\python\ops\script_ops.py", line 221, in __call__
    ret = func(*args)

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 585, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-14-b441973f18d5>", line 20, in generator
    features = np.load(full_path)

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 422, in load
    fid = open(os_fspath(file), "rb")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\STSC.LNVO-126908.000\\Desktop\\FallDetection\\Classes\\Falling\\Falling_477.npy'


	 [[{{node PyFunc}}]] [Op:IteratorGetNextSync

In [31]:
print(tf.__version__)

2.0.0


In [32]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=mylog_dir, update_freq=1000)
print(os.path.join(mylog_dir, 'train'))
#following call works for tensorflow 1.1.4
model.fit(train_dataset, 
          epochs=17, 
          validation_data=valid_dataset, 
          callbacks=[tensorboard_callback])

#model.fit(train_dataset, epochs=17, callbacks=[tensorboard_callback], validation_data=valid_dataset, validation_steps=4, verbose=0)

C:/Users/STSC.LNVO-126908.000/Desktop/FallDetection\train_log\train


UnknownError: FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\STSC.LNVO-126908.000\\Desktop\\FallDetection\\Classes\\Walking\\Walking286.npy'
Traceback (most recent call last):

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\tensorflow_core\python\ops\script_ops.py", line 221, in __call__
    ret = func(*args)

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 585, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-14-b441973f18d5>", line 20, in generator
    features = np.load(full_path)

  File "C:\Users\STSC.LNVO-126908.000\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 422, in load
    fid = open(os_fspath(file), "rb")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\STSC.LNVO-126908.000\\Desktop\\FallDetection\\Classes\\Walking\\Walking286.npy'


	 [[{{node PyFunc}}]] [Op:IteratorGetNextSync]

In [33]:
model.file=os.path.join(BASE_PATH,'my_model.h5')
model.save(model.file)

ValueError: Weights for model sequential have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])


### STEP 12:  save the tensorflow model to an h5 file

In [ ]:
BASE_DATA_PATH = '/Users/subhi/Downloads/CV-Proj3/Data'
model_file = os.path.join(BASE_DATA_PATH, 'my_model.h5')
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model shuold be saved to HDF5.
model.save(model_file) 

### STEP 13: try to conver the model to tflite --- Support to come 2019 (when?)--Curently LSTM conversion to TFLite NOT supported

In [28]:
#from tensorflow import lite
tflite_file  = os.path.join(BASE_DATA_PATH, 'my_tflite_model.tflite')
print(" want to save tflite_file" + tflite_file)
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()




#now save the tflite model to the file
#tflite_model.save(tflite_file)   #Note this does not seem to work although in google documentation
open(tflite_file, "wb").write(tflite_model)

 want to save tflite_file/Users/subhi/Downloads/CV-proj3/my_tflite_model.tflite


ConverterError: See console for info.
/Users/subhi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2019-11-28 09:17:58.493144: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-28 09:17:58.514648: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9f5d4d0410 executing computations on platform Host. Devices:
2019-11-28 09:17:58.514665: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2019-11-28 09:17:58.569638: I tensorflow/lite/toco/import_tensorflow.cc:659] Converting unsupported operation: TensorListFromTensor
2019-11-28 09:17:58.569678: I tensorflow/lite/toco/impor

### STEP 14: run evaluation on the test data feature extraction

In [29]:
# evaluate the test data using model


# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')

# NOTE: should have separate test data but, only have validation data
#results = model.evaluate_generator(val_data_gen, verbose=1)
results = model.evaluate(valid_dataset, verbose=1)
print('test loss, test acc:', results)





# Evaluate on test data
19/19 [==============================] - 3s 139ms/step - loss: 0.0429 - accuracy: 0.9934 - top_k_categorical_accuracy: 1.0000
test loss, test acc: [0.042945761266672, 0.9934211, 1.0]


### STEP 15: Run predictions on the test data feature extracted

In [30]:
# make predictions

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions ')
predictions = model.predict(valid_dataset, verbose=1 )



# Generate predictions 
19/19 [==============================] - 3s 168ms/step


In [31]:
#print out prediction info for validation data set (as do not have separate test data set)
print('predictions shape:', predictions.shape)
print(predictions)
print(len(predictions))


predictions shape: (304, 2)
[[9.99998927e-01 1.03256002e-06]
 [1.67706972e-07 9.99999881e-01]
 [1.00000000e+00 6.82283674e-10]
 [1.11434516e-11 1.00000000e+00]
 [9.99999762e-01 2.29326602e-07]
 [9.99998808e-01 1.16833326e-06]
 [9.99999762e-01 1.82261132e-07]
 [9.99918461e-01 8.15016028e-05]
 [9.99926448e-01 7.35764625e-05]
 [9.99999881e-01 8.47332089e-08]
 [1.00000000e+00 3.23047611e-09]
 [9.99956131e-01 4.38320676e-05]
 [3.47217033e-03 9.96527851e-01]
 [9.99999285e-01 6.98495455e-07]
 [9.99999642e-01 3.91575156e-07]
 [7.44205808e-10 1.00000000e+00]
 [9.99999404e-01 5.40846315e-07]
 [1.52434044e-12 1.00000000e+00]
 [9.99999762e-01 2.48983440e-07]
 [3.14490760e-11 1.00000000e+00]
 [9.33684468e-01 6.63155392e-02]
 [6.11079451e-11 1.00000000e+00]
 [1.48474621e-09 1.00000000e+00]
 [9.99998212e-01 1.83439920e-06]
 [9.99998689e-01 1.30680746e-06]
 [6.44362656e-13 1.00000000e+00]
 [9.99998808e-01 1.13260558e-06]
 [9.99999642e-01 3.91971213e-07]
 [1.00000000e+00 3.20483657e-08]
 [1.94535144e-0